In [3]:
sc##################can we have synonyms in tfidf matrix??????

In [1]:
from pyspark.sql import SQLContext

In [2]:
sqlContext = SQLContext(sc)
#df=sc.textFile('SMALL.JSON')
df = sqlContext.read.json('SMALL.JSON')


In [ ]:
df.printSchema()

In [3]:
df1=df.select("description")

In [4]:
df2=df1.dropDuplicates()

In [10]:
df2.printSchema()

root
 |-- description: string (nullable = true)



In [11]:
doc=df2.rdd.zipWithIndex()

In [20]:
df3=doc.toDF()

In [21]:
df3.printSchema()

root
 |-- _1: struct (nullable = true)
 |    |-- description: string (nullable = true)
 |-- _2: long (nullable = true)



In [22]:
import re
#from stemming.porter2 import stem
def clean_word(w):
    w= str(w)
    w=re.sub('\n',"",w)
    return re.sub("[^a-z| |0-9]|,\,|\.|\;|\:|\;|\?|\!|\[|\]|\}|\{(?i)\b|[0-9]|((?:https?://|www\d{0,3}))|[//]|[#]|[$]|", "", (w.lower()))


In [25]:
text=df3.rdd.map(lambda x: (clean_word(x[0]),x[1])).toDF().withColumnRenamed("_1","doc_text").withColumnRenamed("_2","doc_id")

In [16]:
text=df2.rdd.map(lambda x: (clean_word(x))).toDF().withColumnRenamed("_1","doc_text")

TypeError: Can not infer schema for type: <class 'str'>

In [26]:
df = (text
      .rdd
  .map(lambda x : (x.doc_id,x.doc_text.split(" ")))
  .toDF()
  .withColumnRenamed("_1","doc_id")
  .withColumnRenamed("_2","features"))


In [27]:
df.printSchema()

root
 |-- doc_id: long (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [28]:
from pyspark.ml.feature import StopWordsRemover

In [29]:
remover = StopWordsRemover(inputCol="features", outputCol="filtered")

In [30]:
df1=remover.transform(df)

In [ ]:
df1.show()

In [31]:
from pyspark.ml.feature import HashingTF
from pyspark.ml.feature import IDF
from pyspark.sql.types import DoubleType

In [54]:
htf = HashingTF(inputCol="filtered", outputCol="tf",numFeatures=1000)
tf = htf.transform(df1)


In [55]:
tf.cache()
idf = IDF(inputCol="tf", outputCol="tfidf")
tfidf = idf.fit(tf).transform(tf)


In [34]:
from sklearn.metrics.pairwise import cosine_similarity

In [56]:
mat1=tfidf.rdd.map(lambda x: (x.tfidf))

In [57]:
import numpy as np
from scipy.sparse import csr_matrix
def as_matrix(vec):
    data, indices = vec.values, vec.indices
    shape = 1, vec.size
    return csr_matrix((data, indices, np.array([0, vec.values.size])), shape)

mats = mat1.map(as_matrix)

In [58]:
from scipy.sparse import vstack

mat1 = mats.reduce(lambda x, y: vstack([x, y]))

In [ ]:
mat1

In [59]:
cosimilarities = cosine_similarity(mat1, mat1)

In [60]:
index=np.argsort(-cosimilarities)

In [62]:
for i in range(3):##### change here
    a=(index[i]).tolist()
    df4=doc.filter(lambda x: x[1]==a[0]).map (lambda x: x[0])
    print ('='*60)
    print ('Document Similarity Analysis using Cosine Similarity\n')
    print ('='*60)
    print ("documents:",i+1,":",df4.collect())
   # print (df3.collect())    
    for j in (range(2)):
        df5=doc.filter(lambda x: x[1]==a[j+1]).map (lambda x: x[0])
        print ('-'*100 )
        print (" similar documents\n","Top Doc:", j+1,":\n",df5.collect() )
       # print (df2.collect())
        print ('-'*100 )

Document Similarity Analysis using Cosine Similarity

documents: 1 : [Row(description='Date: 12/10/2010 Case #: 10-12574 Author ID: SRK-241\n\nOn December 2, 2010 the above male party entered Allendale Liquors to cash a check. The suspect\nhanded over a FED-EX check with a Wachovia Bank label. The suspect produced a NJ Temporary\nDrivers License which had been altered. The suspect also showed the employee what appeared to\nbe a FED-EX employee identification card but did not hand it over.\n\nInfo on the Temp NJ DL: Lixander Viveros, 33 Lake St. Mahwah NJ 07446 is a bad address.\n\nAny agency that has similar or could ID suspect is asked to contact P.O. Scott Kuenzel or Det. John\nMattiace.\nAllendale Police Dept. - Headquarters\n(201) 825-1900\n')]
----------------------------------------------------------------------------------------------------
 similar documents
 Top Doc: 1 :
 [Row(description='Date: 5/23/2011 Case #: 11-18434 Author ID: 516\nCraig Avent, DOB/9-25-1976, Rahway N.J.

In [ ]:
tags=text.map(lambda line: line.split(" "))

In [ ]:
from pyspark.mllib.feature import HashingTF

In [ ]:
hashingTF = HashingTF(1000)
tf = hashingTF.transform(tags)

In [ ]:
tf.count()

In [ ]:
from pyspark.mllib.feature import IDF

In [ ]:
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

In [ ]:
tfidf.first()

In [ ]:
from pyspark.ml.linalg import SparseVector
from operator import attrgetter

df = mats.toDF(["features"])



In [ ]:
from pyspark.mllib.linalg.distributed import RowMatrix,IndexedRowMatrix

In [ ]:
mat=RowMatrix (tfidf)

In [ ]:
rows = mat.rows

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
def as_matrix(vec):
    data, indices = vec.values, vec.indices
    shape = 1, vec.size
    return csr_matrix((data, indices, np.array([0, vec.values.size])), shape)

mats = rows.map(as_matrix)

In [ ]:
mats.count()

In [ ]:
from scipy.sparse import vstack

mat1 = mats.reduce(lambda x, y: vstack([x, y]))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosimilarities = cosine_similarity(mat1, mat1)

In [ ]:
cosimilarities.shape